# Working with Geolocated Data
---

### Exercise: Earthquake Monitor

Collect the data:

In [1]:
import requests

endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "csv", 
       "starttime": "2023-08-28", 
       "endtime": "2023-09-28",
       "minmagnitude": 5
     }

response = requests.get(endpoint, params=ps)

import pandas as pd
import io

buf = io.StringIO(response.text) 
data = pd.read_csv(buf)

The available information about each earthquake:

In [2]:
for col in data.columns:
    print(col)

time
latitude
longitude
depth
mag
magType
nst
gap
dmin
rms
net
id
updated
place
type
horizontalError
depthError
magError
magNst
status
locationSource
magSource


Draw the map:

In [3]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for i in range(len(data)):
    quake = data.iloc[i,:]
    folium.Marker(location=[quake.latitude,quake.longitude], 
                  popup = quake.mag
                 ).add_to(world_map)


world_map

Again but with `folium.CircleMarker`:

In [4]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for i in range(len(data)):
    quake = data.iloc[i,:]
    folium.CircleMarker(location=[quake.latitude,quake.longitude], 
                        popup = quake.mag,
                        weight=1,
                        fill=True,
                        fill_opacity=0.3,
                        radius = quake.mag * 2
                       ).add_to(world_map)


world_map

For the colouring, we need to request the GeoJSON data:

In [5]:
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "geojson", 
       "starttime": "2023-08-28", 
       "endtime": "2023-09-28",
       "minmagnitude": 5 
     }

response = requests.get(endpoint, params=ps)

import json
data_dict = json.loads(response.text)


Let's inspect the first earthquake record:

In [6]:
quake = data_dict['features'][0]
quake

{'type': 'Feature',
 'properties': {'mag': 5.4,
  'place': 'off the west coast of northern Sumatra',
  'time': 1695836841700,
  'updated': 1695844323624,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000lb60',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000lb60&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': 0,
  'alert': 'green',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 449,
  'net': 'us',
  'code': '6000lb60',
  'ids': ',usauto6000lb60,pt23270000,us6000lb60,',
  'sources': ',usauto,pt,us,',
  'types': ',internal-moment-tensor,internal-origin,losspager,origin,phase-data,shakemap,',
  'nst': 81,
  'dmin': 4.472,
  'rms': 0.72,
  'gap': 58,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 5.4 - off the west coast of northern Sumatra'},
 'geometry': {'type': 'Point', 'coordinates': [93.1233, 0.874, 10]},
 'id': 'us6000lb60'}

Notice that the position is given by

In [7]:
quake['geometry']['coordinates']

[93.1233, 0.874, 10]

which is in the format [longitude, latitude, depth].

The other information can be retrieved as e.g.

In [8]:
quake['properties']['mag']

5.4

In [9]:
import folium

greenwich_coords = [51.4769, 0]  # i.e. [Latitude, Longitude]

world_map = folium.Map(location=greenwich_coords,
                                zoom_start = 1)

# Add markers
for quake in data_dict['features']:
    coords = quake['geometry']['coordinates']
    folium.CircleMarker(location=[coords[1],coords[0]], 
                        popup = quake['properties']['mag'],
                        weight=1,
                        fill=True,
                        fill_opacity=0.3,
                        color=quake['properties']['alert'],
                        radius = quake['properties']['mag'] * 2
                       ).add_to(world_map)


world_map